# Advent of Code Puzzles
[Advent of Code 2019](https://adventofcode.com/2019) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

## [Day 03](https://adventofcode.com/2019/day/3): Crossed Wires 
### Part I
- **Unknown**: The [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) from the central port to the closest intersection.
- **Data**: Two wire's paths
- **Condition**:
    - A wire crossing with itself doesn't count.
- **Plan**:
    - Parse paths to get two lists of segments. Each segment is a double of an axis (category) and a vector (positive or negative number). > `parse()`
    - Calculate grid shape and determine the central port position. > `calculate_limits()` &
    ```python
        # Calculate extent
        shapes = np.array([calculate_limits(wires[0]), calculate_limits(wires[1])])
        extent = [shapes.min(axis=0)[:,0], shapes.max(axis=0)[:,1]]

        # Initialize central port (start) and grids
        start = np.abs(extent[0])
        shape = start + extent[1] + [2,2]
        g1 = np.zeros(shape).astype(int)
        g2 = np.zeros(shape).astype(int)
    ```
    - Trace both wires separately using segments > `trace_wire()` & `write_segm()`
    - Add both grids > `map_wires()`
    - Find all intersections (where value == 2)
    - Calculate the Manhatan distances from start and return the minimum distance > `solve_part1()`



In [2]:
def get_xy(segm):
    """Transform segments to an axis (cat) + a vector (num)."""
    cat = segm[0]
    num = int(segm[1:])
    if cat in ['R', 'L']:
        ax = 'x'
        di = num if cat == 'R' else -num
    elif cat in ['U','D']:
        ax = 'y'
        di = num if cat == 'U' else -num
    else:
        ax, di = cat, num
        print(f"get_xy() warning: An unknown segment {segm}")

    return (ax, di)

for segm in ['R11', 'U12', 'L20', 'D2', 'X12']:
    print(get_xy(segm))

('x', 11)
('y', 12)
('x', -20)
('y', -2)
get_xy() warning: An unknown segment X12
('X', 12)


In [3]:
def parse(file):
    """Parse input to two list with segments (axis, direction)"""
    txt = Path(file).read_text().strip()
    wires = [line.split(',') for line in txt.split('\n')]
    for wire in wires:
        for segm in range(len(wire)):
            wire[segm] = get_xy(wire[segm])
    return wires
wires1 = parse('example1.txt')
print(wires1)
wiresx = parse('input.txt')
print(len(wiresx[0]), len(wiresx[1]))

[[('x', 8), ('y', 5), ('x', -5), ('y', -3)], [('y', 7), ('x', 6), ('y', -4), ('x', -4)]]
301 301


In [4]:
def calculate_limits(wire):
    """Calculate limits of a wire topology."""
    # Initialize limits = [xmin, xmax, ymin, ymax]
    limits = [0, 0, 0, 0]
    # Initialize a variable for tracing position
    coords = [0, 0]
    # Trace the wire
    for ax, di in wire:
        # Update a position and check limits
        if ax == 'x':
            coords[0] += di
            limits[0] = min(limits[0], coords[0])
            limits[1] = max(limits[1], coords[0])
        else:
            coords[1] += di
            limits[2] = min(limits[2], coords[1])
            limits[3] = max(limits[3], coords[1])
    return np.array(limits).reshape((2,2))

for wires in [wires1, wiresx]:
    for wire in wires:
        print(calculate_limits(wire))

[[0 8]
 [0 5]]
[[0 6]
 [0 7]]
[[-4937  2909]
 [-1184 10352]]
[[-8412     0]
 [-5080  3210]]


In [5]:
def write_segm(grid, start, segm):
    """Place the segment to the grid"""
    ax, di = segm
    sx, sy = start 
    if ax == 'x' and di > 0:
        for idx in range(1, di+1):
            grid[sx + idx, sy] = 1
    elif ax == 'x' and di < 0:
        for idx in range(1, np.abs(di)+1):
            grid[sx - idx, sy] = 1
    elif ax == 'y' and di > 0:
        for idx in range(1, di+1):
            grid[sx, sy + idx] = 1
    elif ax == 'y' and di < 0:
        for idx in range(1, np.abs(di)+1):
            grid[sx, sy - idx] = 1
    else:
        print(f"write_segm() warning: something got wrong with start={start} and segm={segm}")
    
    return 

def trace_wire(grid, start, wire):
    """Write segments of the wire to the grid"""

    pos = start.copy() # To ensure same start for both grids
    for segm in wire:
        write_segm(grid,pos,segm)
        ax, di = segm
        if ax == 'x':
            pos[0] += di
        else:
            pos[1] += di
        # print(f"segm={segm}, pos={pos}")
    return

In [6]:
def map_wires(input):
    """Return the grid with wires mapped."""

    # Read input
    wires = parse(input)
    
    # Calculate extent
    shapes = np.array([calculate_limits(wires[0]), calculate_limits(wires[1])])
    extent = [shapes.min(axis=0)[:,0], shapes.max(axis=0)[:,1]]

    # Initialize grids
    start = np.abs(extent[0])
    shape = start + extent[1] + [2,2]
    g1 = np.zeros(shape).astype(int)
    g2 = np.zeros(shape).astype(int)
    print(g1.shape, g2.shape, start)

    # Trace wires
    trace_wire(g1, start, wires[0])
    assert start.all() == np.abs(extent[0]).all()
    trace_wire(g2, start, wires[1])

    # Add grids
    return g1 + g2, start

def solve_part1(input):
    """Solve part 1"""

    out, start = map_wires(input)

    # Find all intersections
    x, y = np.asarray(out == 2).nonzero()
    intersections = np.array(list(zip(x,y)))
 
    return min([x+y for x,y in np.abs(intersections-start)])

In [7]:
assert solve_part1('example1.txt') == 6
assert solve_part1('example2.txt') == 159
assert solve_part1('example3.txt') == 135
assert solve_part1('input.txt') == 2050

(10, 9) (10, 9) [0 0]
(240, 149) (240, 149) [ 0 30]
(181, 122) (181, 122) [ 0 16]
(11323, 15434) (11323, 15434) [8412 5080]


True


### Part II
- **Unknown**: The sum of the fewest combined steps both wires must take to reach an intersection.
- **Data**: Same as in part I.
- **Condition**:
    - If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.
- **Plan**:


In [8]:
def get_grid_intersections(input):
    """Return mapped wires, the central port position and intersections"""

    out, start = map_wires(input)

    # Find all intersections
    x, y = np.asarray(out == 2).nonzero()
    intersections = np.array(list(zip(x,y)))
 
    return out, start, intersections

out, cp, isons = get_grid_intersections('example1.txt')
out, cp, isons

(10, 9) (10, 9) [0 0]


(array([[0, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 0, 1, 0],
        [1, 0, 1, 2, 1, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 1, 2, 1, 1, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([0, 0]),
 array([[3, 3],
        [6, 5]], dtype=int64))